# Produce Hourly Shaped Hydro Profile for the Texas Interconnection

In [1]:
import pandas as pd
from numpy.testing import assert_almost_equal

from powersimdata.scenario.scenario import Scenario
from powersimdata.input.grid import Grid

In [2]:
s = Scenario("87")
hydro_scenario = s.state.get_hydro()

SCENARIO: base | WesternBase_2016_noHVDC_Final_2019Sep

--> State
analyze
--> Loading grid
Loading bus
Loading plant
Loading heat_rate_curve
Loading gencost_before
Loading gencost_after
Loading branch
Loading sub
Loading bus2sub
--> Loading ct
Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv
--> Loading hydro


Create an instance of the current grid

In [3]:
grid = Grid(["Texas"])

Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv


In [4]:
hydro_v2 = pd.DataFrame(index = hydro_scenario.index, columns=grid.plant[grid.plant['type'] == 'hydro'].index)

In [5]:
filename = "../../data/texas_hydro_generation.csv"
hydro_profile_ercot = pd.read_csv(filename, header=0, index_col=0, parse_dates=True, squeeze=True)

In [6]:
total_hydro_capacity = grid.plant[grid.plant["type"] == "hydro"]["Pmax"].sum()
for i in hydro_v2.columns:
    cap = grid.plant.loc[i]["Pmax"]
    hydro_v2[i] = [g * cap / total_hydro_capacity for g in hydro_profile_ercot]

In [7]:
hydro_v2

plant_id,12977,12978,12997,12998,13104,13141,13159,13160,13182,13198,...,13201,13202,13203,13204,13205,13206,13461,13462,13463,13464
UTC,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,7.263016,3.019456,5.875698,1.566853,0.587570,2.774635,10.657863,10.657863,11.327040,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,13.057107,12.371609,3.607026,7.834264
2016-01-01 01:00:00,6.325077,2.629526,5.116916,1.364511,0.511692,2.416321,9.281517,9.281517,9.864277,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.370924,10.773951,3.141218,6.822554
2016-01-01 02:00:00,3.150513,1.309764,2.548730,0.679661,0.254873,1.203567,4.623113,4.623113,4.913385,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.663844,5.366493,1.564637,3.398307
2016-01-01 03:00:00,1.410917,0.586561,1.141416,0.304378,0.114142,0.539002,2.070402,2.070402,2.200396,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.536480,2.403315,0.700703,1.521888
2016-01-01 04:00:00,1.146370,0.476581,0.927400,0.247307,0.092740,0.437939,1.682201,1.682201,1.787822,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.060890,1.952693,0.569321,1.236534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-31 19:00:00,5.061934,2.104399,4.095048,1.092013,0.409505,1.933773,7.427961,7.427961,7.894342,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.100106,8.622350,2.513904,5.460064
2016-12-31 20:00:00,5.079701,2.111786,4.109421,1.095846,0.410942,1.940560,7.454034,7.454034,7.922051,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.132047,8.652615,2.522728,5.479228
2016-12-31 21:00:00,5.145325,2.139068,4.162510,1.110003,0.416251,1.965630,7.550330,7.550330,8.024394,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.250022,8.764396,2.555319,5.550013


In [8]:
hydro_v2.to_csv('texas_hydro_v2_reference.csv')

In [9]:
assert_almost_equal(hydro_v2.sum().sum(), hydro_profile_ercot.sum(), decimal=5)